In [94]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [174]:
import torch.nn as nn

# the following neural network is to model the value function
class MyNet(nn.Module):
    def __init__(self, m, n):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(1 + m * n, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

# x is 2 dim torch tensor with shape (m, n)
    def forward(self, t, x):
        #make x 1 dim
        x = x.view(-1)
        x = torch.cat((t, x), dim=0)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    
# The Gaussian policy returns torch tensor with shape (m, n)
def Gaussian_Policy(net, t, q, A, B, gamma):
    
    bid_matrix  = torch.zeros(q.shape)
    ask_matrix  = torch.zeros(q.shape)

    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            x = torch.zeros(q.shape)
            x[i, j] = 1
            mean_bid = A[i, j] / (2 * B[i, j]) + 0.5 * (net.forward(t, q) - net.forward(t, q + x))
            mean_ask = A[i, j] / (2 * B[i, j]) + 0.5 * (net.forward(t, q) - net.forward(t, q - x))
            variance = gamma / (2 * B[i, j])
            std = torch.sqrt(variance)
            bid_matrix[i, j] = torch.normal(mean_bid, std)
            ask_matrix[i, j] = torch.normal(mean_ask, std)

    return bid_matrix, ask_matrix
    


In [214]:
import math
import scipy.stats as scistat

def Market_Order_Simulation(dt, A, B, bid_matrix, ask_matrix):
    orders = torch.zeros(bid_matrix.shape)
    buy_orders = torch.zeros(bid_matrix.shape)
    sell_orders = torch.zeros(bid_matrix.shape)
    m = bid_matrix.shape[0]
    n = bid_matrix.shape[1]
    for i in range(m):
        for j in range(n):
            intensity_bid = A[i, j] - B[i, j] * bid_matrix[i, j]
            intensity_ask = A[i, j] - B[i, j] * ask_matrix[i, j]
            buy_orders[i, j] = torch.poisson(intensity_bid * dt)
            sell_orders[i, j] = torch.poisson(intensity_ask * dt)
            orders[i, j] = buy_orders[i, j] - sell_orders[i, j]

    return [orders, buy_orders, sell_orders]

# this function returns the inventory path
def Train_Data_Simulation(T, dt, A, B, gamma, net):
    N = int(T / dt)
    m = A.shape[0]
    n = A.shape[1]
    q = torch.zeros((N, m, n))
    buy = torch.zeros((N, m, n))
    sell = torch.zeros((N, m, n))

    for count in range(N - 1):
        t = torch.Tensor([count * dt])
        bid_matrix, ask_matrix = Gaussian_Policy(net, t, q[count], A, B, gamma)
        orders = Market_Order_Simulation(dt, A, B, bid_matrix, ask_matrix)[0]
        buy[count] = Market_Order_Simulation(dt, A, B, bid_matrix, ask_matrix)[1]
        sell[count] = Market_Order_Simulation(dt, A, B, bid_matrix, ask_matrix)[2]
        q[count + 1] = q[count] + orders
    
    return q, buy, sell

# this function returns the stock prices path
def Stock_Prices_Simulation(T, dt, mu, sigma, S0):
    N = int(T / dt)
    S = torch.zeros(N)
    S[0] = S0
    for count in range(N - 1):
        S[count + 1] = S[count] + mu * S[count] * dt + sigma * S[count] * math.sqrt(dt) * torch.normal(0.0, 1.0, size=(1,))
    return S

    

In [156]:


#the following are option greeks' functions
def d1(S, K, r, sigma, T):
    return (np.log(S/K) + (r+sigma*sigma/2)*T)/(sigma*np.sqrt(T))

def d2(S, K, r, sigma, T):
    return d1(S, K, r, sigma, T) - sigma*np.sqrt(T)

def Gamma(S, K, r, sigma, T):
    return scistat.norm.pdf(d1(S, K, r, sigma, T))/(S*sigma*np.sqrt(T))

def Theta(S, K, r, sigma, T):
    aux1 = -S*scistat.norm.pdf(d1(S, K, r, sigma, T))*sigma/(2*np.sqrt(T))
    aux2 = -r*K*np.exp(-r*T)*scistat.norm.cdf(d2(S, K, r, sigma, T))
    return aux1+aux2


def Options_Theta(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes):
    N = int(T / dt)
    m = Vol_surface.shape[0]
    n = Vol_surface.shape[1]
    theta = torch.zeros((N, m, n))
    S = Stock_Prices_Simulation(T, dt, mu, sigma, S0)
    for count in range(N):
        t = count * dt
        for i in range(m):
            for j in range(n):
                theta[count, i, j] = Theta(S[count], Strikes[i], r, Vol_surface[i, j], Maturities[j] - t)
    return theta


def Options_Gamma(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes):
    N = int(T / dt)
    m = Vol_surface.shape[0]
    n = Vol_surface.shape[1]
    gamma = torch.zeros((N, m, n))
    S = Stock_Prices_Simulation(T, dt, mu, sigma, S0)
    for count in range(N):
        t = count * dt
        for i in range(m):
            for j in range(n):
                gamma[count, i, j] = Gamma(S[count], Strikes[i], r, Vol_surface[i, j], Maturities[j] - t)
    return gamma
    

In [253]:
def option_loss(Vol_surface, S0, q, mu, sigma, Strikes, Maturities, r, T, dt):
    N = int(T / dt)
    m = Vol_surface.shape[0]
    n = Vol_surface.shape[1]
    loss = torch.zeros(N)
    for count in range(N):
        t = count * dt
        loss[count] =  torch.sum((Options_Theta(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes)[count] + 0.5 * Options_Gamma(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes)[count]) * q[count]) 
    return loss


def inventory_loss(A, B, q, buy, sell, net, T, dt):
    N = int(T / dt)
    m = A.shape[0]
    n = A.shape[1]
    loss = torch.zeros(N)
    for count in range(N):
        t = torch.tensor([count * dt])
        loss1 = torch.sum((A / (2 * B)) * (buy[count] + sell[count])) + 0.5 * net.forward(t, q[count]) * torch.sum(buy[count] + sell[count]) 
        loss2 = torch.tensor([0.0])
        for i in range(m):
            for j in range(n):
                x = torch.zeros((m, n))
                x[i, j] = 1
                loss2 += net.forward(t, q[count] + x) * buy[count, i, j] + net.forward(t, q[count] - x) * sell[count, i, j]
        loss[count] = loss1 - loss2
        
    return loss

def value_function_loss(T, dt, q, net):
    N = int(T / dt)
    dt = torch.tensor([dt])
    loss = torch.zeros(N)
    for count in range(N - 1):
        t = torch.tensor([count * dt])
        loss[count] = (net.forward(t + dt, q[count + 1]) - net.forward(t, q[count])) / dt
    return loss


def constant_loss(gamma, B, T, dt):
    N = int(T / dt)
    m = B.shape[0]
    n = B.shape[1]
    loss = torch.zeros(N)
    for count in range(N):
        loss[count] = gamma * ((m * n * 1.79817986835) + torch.sum(torch.log(gamma / (2 * B))))
    return loss

def total_loss(Vol_surface, S0, q, mu, sigma, Strikes, Maturities, r, T, dt, A, B, buy, sell, net, gamma):
    loss1 = option_loss(Vol_surface, S0, q, mu, sigma, Strikes, Maturities, r, T, dt)[:-1]
    loss2 = inventory_loss(A, B, q, buy, sell, net, T, dt)[: -1]
    loss3 = value_function_loss(T, dt, q, net)[:-1]
    loss4 = constant_loss(gamma, B, T, dt)[:-1]
    loss = 0.5 * dt * torch.sum((loss1 + loss2 + loss3 - loss4)**2)
    return loss



In [254]:
Vol_surface = torch.tensor([[0.2, 0.3], [0.4, 0.5]])
S0 = 100
r = 0.05
T = 1
dt = 0.01
mu = 0.1
sigma = 0.2
gamma = 0.05
Maturities = torch.tensor([3, 5])
Strikes = torch.tensor([90, 110])
net = MyNet(2, 2)

A = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
B = torch.tensor([[0.2, 0.3], [0.2, 0.5]])

theta_matrix = Options_Theta(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes)
gamma_matrix = Options_Gamma(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes)
q, buy, sell = Train_Data_Simulation(T, dt, A, B, gamma, net)





In [255]:
loss = total_loss(Vol_surface, S0, q, mu, sigma, Strikes, Maturities, r, T, dt, A, B, buy, sell, net, gamma)

In [257]:
loss

tensor(65.2333, grad_fn=<MulBackward0>)

In [184]:
#let t be a tensor scalar
t = torch.tensor([1.0])
x = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
# set random seed 
torch.manual_seed(0)
net = MyNet(2, 2)

# test the Gaussian policy
A = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
B = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
q = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
gamma = 0.1
bid_matrix, ask_matrix = Gaussian_Policy(net, t, q, A, B, gamma)

print(bid_matrix)
print(ask_matrix)

tensor([[0.3375, 0.2461],
        [0.4124, 0.5485]], grad_fn=<CopySlices>)
tensor([[0.1838, 0.6574],
        [0.4161, 0.5759]], grad_fn=<CopySlices>)
